In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertConfig, BertForSequenceClassification, BertTokenizer, BertConfig
from datasets import load_dataset,load_metric
import numpy as np


from accelerate import Accelerator


accelerator = Accelerator()
device = accelerator.device

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
texts = "I don't like this movie."

# pretrained_name = "distilbert-base-uncased-finetuned-sst-2-english"
pretrained_name = "textattack/bert-base-uncased-SST-2"

# pred_config = DistilBertConfig.from_pretrained(pretrained_name)
# pred_tokenizer = DistilBertTokenizer.from_pretrained(pretrained_name)
# pred_model = DistilBertForSequenceClassification.from_pretrained(pretrained_name).to(device)

pred_config = BertConfig.from_pretrained(pretrained_name)
pred_tokenizer = BertTokenizer.from_pretrained(pretrained_name)
pred_model = BertForSequenceClassification.from_pretrained(pretrained_name).to(device)

inputs = pred_tokenizer(texts, return_tensors="pt")
with torch.no_grad():
    inputs = {key:val.to(device) for key,val in inputs.items()}
    logits = pred_model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(pred_model.config.id2label[predicted_class_id])

print(inputs)

LABEL_0
{'input_ids': tensor([[ 101, 1045, 2123, 1005, 1056, 2066, 2023, 3185, 1012,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [3]:
# from datasets import load_dataset
# imdb = load_dataset("imdb")


In [4]:
from maskgen.models.random_mask import RandomMaskSaliency
random_mask = RandomMaskSaliency(pred_model, 2)



In [7]:
# texts = "The incoherent drama Browse tries to look like a suspenseful horror movie, but there's nothing scary or thrilling about this rambling dud of a film."
# texts = "A great deal of the movie consists of unfinished ideas, plot points, story threads, and character arcs."
# texts = "It's a thoughtful, curious piece of work which may not quite be successful in finding its destination but which manages to intrigue along the way."
# texts = "This thriller doesn't thrill. It occasionally amuses, and it rarely makes any real sense."
# texts = "Skip the film if you need an ending that answers the questions raised throughout. But check it out if you enjoy seeing a psychological thriller that makes you come to your own conclusions."

# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
# texts = "I really love this film."
# texts = 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have to always say "Gene Roddenberry\'s Earth..." otherwise people would not continue watching. Roddenberry\'s ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.'

# idx = 14
# texts = imdb["test"][idx]['text']

label = None
# label = torch.tensor([1]).to(device)

inputs = pred_tokenizer(texts, return_tensors="pt")
with torch.no_grad():
    inputs = {key:val.to(device) for key,val in inputs.items()}
    logits = pred_model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(pred_model.config.id2label[predicted_class_id])

if label is None:
    label = logits.argmax().unsqueeze(0)


from captum.attr import visualization
# expl = mask_gen_model.attribute_text(inputs['input_ids'], inputs['attention_mask'], label)[0][1:-1]
# random attribution
# expl = torch.randn_like(inputs['input_ids'].float(), requires_grad=False)[0][1:-1]
expl = random_mask.attribute_text(inputs['input_ids'], inputs['attention_mask'], n_samples=500)[0][1:-1]
print(expl)
tokens = pred_tokenizer.convert_ids_to_tokens(inputs['input_ids'].flatten())[1:-1]
# normalize expl
expl = (expl - expl.min()) / (expl.max() - expl.min()) #- 0.5
vis_data_records = [visualization.VisualizationDataRecord(
                                 label * expl + (1 - label) * (-expl),
                                # expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
visualization.visualize_text(vis_data_records)

LABEL_0
correct_idx torch.Size([500, 1])
probs torch.Size([500, 1])
weighted_mask torch.Size([1, 25])
tensor([0.4937, 0.5675, 0.5265, 0.5687, 0.5415, 0.5314, 0.5083, 0.5111, 0.4990,
        0.4974, 0.4577, 0.4576, 0.4497, 0.5440, 0.4198, 0.5048, 0.5072, 0.4669,
        0.4844, 0.4937, 0.5348, 0.6727, 0.4821], device='cuda:0')


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,1.00,"i really didn ' t like this movie . some of the actors were good , but overall the movie was boring ."


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,1.00,"i really didn ' t like this movie . some of the actors were good , but overall the movie was boring ."


In [6]:
label

tensor([1], device='cuda:0')

In [21]:
expl.shape

torch.Size([21])

In [ ]:
len(tokens)

23